<a href="https://colab.research.google.com/github/Taedriel/ZSL-v2/blob/wordEmbedding/WordsEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Kingston/ZSL-v2/

/content/drive/MyDrive/Kingston/ZSL-v2


In [ ]:
!pip install transformers wget 

In [ ]:
import wget, os

import torch
from transformers import BertTokenizer, BertModel

import logging
logging.basicConfig(level = logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
logging.info("Downloading dataset...")

url = ""

INFO:root:Downloading dataset...


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', padding=True, truncation=True,)

model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states = True, )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tag = "cat"

inputs = tokenizer(tag, return_tensors = "pt")

with torch.no_grad():
    outputs = model(**inputs)

In [ ]:
hidden_states = outputs[2]

print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

# We got:
# [# layers, # batches, # tokens, # features]
# Desired dimensions:
# [# tokens, # layers, # features]

# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(hidden_states, dim=0)
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)
# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

# apply different strategy to summarize word embeddings
token_sum_4 = []
for token in token_embeddings:

    sum_vec = torch.sum(token[-4:], dim = 0)
    token_sum_4.append(sum_vec)

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 3
Number of hidden units: 768


In [ ]:
print(token_embeddings[1])

print(tag, list(token_sum_4[1][:10]))

tensor([[ 0.6106, -0.5393, -0.0042,  ...,  0.3292,  1.0776,  1.1441],
        [ 0.7220, -0.3811,  0.7036,  ...,  0.1724,  0.6989,  1.1222],
        [ 0.1766, -0.7403,  1.2579,  ...,  0.2124,  0.5306,  0.9950],
        ...,
        [-0.3763, -0.1390, -0.1795,  ..., -0.2885,  0.0480,  0.1925],
        [-0.3609, -0.0988,  0.0371,  ..., -0.4235,  0.2164,  0.1357],
        [-0.1920, -0.0516,  0.1069,  ..., -0.0317,  0.3776,  0.0912]])
cat [tensor(-1.3408), tensor(-0.2139), tensor(-0.1473), tensor(-3.1636), tensor(0.3934), tensor(1.1316), tensor(2.1431), tensor(0.8657), tensor(-1.4748), tensor(-3.9480)]


In [ ]:
!git status

On branch wordEmbeddings
Your branch is up to date with 'origin/wordEmbeddings'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Utils.ipynb
	modified:   WordsEmbeddings.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
